In [21]:
import os
import pandas as pd

edges = pd.read_csv(os.path.join(os.pardir, 'data', 'edgelist.txt'), sep='\t').rename(columns={'1001': 'src', '9304045': 'trg'})
edges.head()

,src,trg
0,1001,9308122
1,1001,9309097
2,1001,9311042
3,1001,9401139
4,1001,9404151


In [ ]:
import sys
scripts_dir = os.path.join(os.path.dirname(os.path.abspath('')), 'scripts')
if not scripts_dir in sys.path:
    sys.path.append(scripts_dir)

NameError: name 'nx' is not defined

In [57]:
import networkx as nx
from get_graph import get_digraph

g: nx.DiGraph = get_digraph()

In [53]:
import dgl

G = dgl.from_networkx(g)

In [ ]:
import numpy as np
word_embs = np.load(os.path.join(os.pardir, 'embeddings.npy'))

27373

In [ ]:
import torch
G.ndata['word_embs'] = torch.from_numpy(np.vstack([word_embs, np.zeros((len(g.nodes()) - len(word_embs), 50))])).float()

In [72]:
from train_test_split import make_split

adj_train, train_edges, train_edges_false, \
        val_edges, val_edges_false, test_edges, test_edges_false = make_split(nx.to_scipy_sparse_array(g))

In [89]:
train_edges = torch.cat([torch.as_tensor(train_edges), torch.as_tensor(train_edges_false)])

train_u = train_edges[:, 0]
train_v = train_edges[:, 1]
train_label = torch.cat([torch.zeros(len(train_edges)), torch.ones(len(train_edges_false))])

In [78]:
test_edges = torch.cat([torch.as_tensor(test_edges), torch.as_tensor(test_edges_false)])

test_u = test_edges[:, 0]
test_v = test_edges[:, 1]
test_label = torch.cat([torch.zeros(len(test_edges)), torch.ones(len(test_edges_false))])

In [79]:
from gat import GraphSAGE

net = GraphSAGE(50, 64)

In [90]:
import torch.nn.functional as F
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)

all_logits = []
for e in range(100):
    # forward
    logits = net(G, G.ndata['word_embs'].float())
    print(logits.shape)
    pred = torch.sigmoid((logits[train_u] * logits[train_v]).sum(dim=1))
    print(pred.shape, train_v.shape, train_u.shape, train_label.shape)
    # compute loss
    loss = F.binary_cross_entropy(pred, train_label)
    
    # backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    all_logits.append(logits.detach())
    
    if e % 5 == 0:
        print('In epoch {}, loss: {}'.format(e, loss))

torch.Size([27770, 64])
torch.Size([106104]) torch.Size([106104]) torch.Size([106104]) torch.Size([141472])


ValueError: Using a target size (torch.Size([141472])) that is different to the input size (torch.Size([106104])) is deprecated. Please ensure they have the same size.